In [1]:
!pip install yfinance

In [2]:
!pip install plotly

In [3]:
!pip install --upgrade nbformat

In [4]:
!pip install --upgrade fbprophet

  Using cached cmdstanpy-0.9.5-py3-none-any.whl (37 kB)
  Using cached setuptools_git-1.2-py2.py3-none-any.whl (10 kB)


In [5]:
!conda install -c conda-forge fbprophet

Solving environment: ...working... done

# All requested packages already installed.



Did not find path entry C:\Miniconda3\bin


In [6]:
# load libraries
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
#import warnings
#warnings.filterwarnings('ignore')
#pd.options.display.float_format = '${:,.2f}'.format

In [7]:
symbol = 'ETH-USD'
today = datetime.today().strftime('%Y-%m-%d')
start_date = '2016-01-01'
df_coin = yf.download(symbol, start_date, today)
df_coin.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-16,3330.387207,3376.401123,3291.563721,3350.921875,3350.921875,9505934874
2022-01-17,3350.947266,3355.819336,3157.224121,3212.304932,3212.304932,12344309617
2022-01-18,3212.287598,3236.016113,3096.123535,3164.025146,3164.025146,13024154091
2022-01-19,3163.850342,3171.158447,3055.212402,3095.825928,3095.825928,13187424144
2022-01-20,3095.062744,3262.418701,3093.455566,3244.378906,3244.378906,8948327424


In [8]:
df_coin.reset_index(inplace=True)
df_coin.columns 
df = df_coin[["Date", "Open"]]
new_names = {
    "Date": "ds", 
    "Open": "y",
}
df.rename(columns=new_names, inplace=True)

C:\Miniconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [9]:
import logging
logging.getLogger('fbprophet').setLevel(logging.WARNING) 
from tqdm.notebook import tqdm

In [11]:
# set the training size - full set
df_train_sweep = df.copy()
print('full train set size = {}'.format(df_train_sweep.shape[0]))

# making the training set smaller to avoid overfit
first_cutoff_day = '2021-07-01'
for start_day in tqdm(['2016-01-01', '2017-01-01', '2018-01-01', '2019-01-01', '2020-01-01', '2021-01-01']):
    df_train_sweep = df[df['ds'] >= start_day]
    init_train_size = df[(df['ds'] >= start_day)&(df['ds'] < first_cutoff_day)].shape[0]
    print('start day {}: train set size = {}'.format(start_day,init_train_size))
    init_train_size_s = '{} days'.format(init_train_size)
    print(init_train_size_s)

full train set size = 1534


1330 days
1330 days
1277 days
912 days
547 days
181 days



In [11]:
from fbprophet.plot import plot_cross_validation_metric
from fbprophet.diagnostics import performance_metrics
from fbprophet.diagnostics import cross_validation

In [21]:
# hyper - parameters tunning
# facebook.github.io/prophet/docs/diagnostics.html#hyperparameter-tuning

# Python
import itertools
import numpy as np

param_grid = {  
    'changepoint_prior_scale': [0.01, 0.1, 0.5, 10, 100, 1000],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
    'holidays_prior_scale': [0.01, 10],
    'seasonality_mode': ['additive', 'multiplicative'],
    'changepoint_range': [0.8, 0.85, 0.9, 0.95, 0.99]
}

# #test
# param_grid = {  
#     'changepoint_prior_scale': [0.01],
#     'seasonality_prior_scale': [0.01],
#     'holidays_prior_scale': [0.01],
#     'seasonality_mode': ['multiplicative'],
#     'changepoint_range': [0.8]
# }

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here
mapes = []  # Store the MAPEs for each params here
mdapes = []  # Store the MDAPEs for each params here
tuning_results_all = pd.DataFrame()

# 1. loop over training set sizes
# for start_day in tqdm(['2016-01-01', '2021-01-01'], leave=True, desc='1st loop'): # test
# for start_day in tqdm(['2016-01-01', '2017-01-01', '2018-01-01', '2019-01-01', '2020-01-01', '2021-01-01'], leave=True, desc='1st loop'): # full
for start_day in tqdm(['2016-01-01', '2018-01-01', '2020-01-01', '2021-01-01'], leave=True, desc='1st loop'): # limited
    df_train_sweep = df[df['ds'] >= start_day]
    init_train_size = df[(df['ds'] >= start_day)&(df['ds'] < first_cutoff_day)].shape[0]
    init_train_size_s = '{} days'.format(init_train_size)

    # 2. Use cross validation to evaluate all parameters
    #for params in all_params:
    for params in tqdm(all_params, leave=False, desc='2nd loop'):
        m_sweep = Prophet(**params).fit(df)  # Fit model with given params
        df_cv = cross_validation(m_sweep, initial=init_train_size_s, period='5 days', horizon = '10 days', parallel='processes')
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])
        mapes.append(df_p['mape'].values[0])
        mdapes.append(df_p['mdape'].values[0])

    # Find the best parameters
    tuning_results = pd.DataFrame(all_params)
    tuning_results['rmse'] = rmses
    tuning_results['mape'] = mapes
    tuning_results['mdape'] = mdapes
    tuning_results['init_train_size'] = init_train_size
    tuning_results_all = tuning_results_all.append(tuning_results,ignore_index=True)
    
    #print(tuning_results)
    best_params = all_params[np.argmin(mapes)]
    #print('For start day {} best (MAPE) parameters are: {}'.format(start_day,best_params))

    # reset the lists for next training size iteration
    rmses = []
    mapes = []
    mdapes = []

1st loop:   0%|          | 0/4 [00:00<?, ?it/s]

2nd loop:   0%|          | 0/480 [00:00<?, ?it/s]

In [ ]:
tuning_results_all.head(10)

In [ ]:
tuning_results_all.to_csv('fbprophet_sweep_full.csv', index=False, mode='a', header=True) # ? first time add header

In [ ]:
tuning_results_all.iloc[tuning_results_all['mape'].argmin()]

In [ ]:
tuning_results_all.iloc[tuning_results_all['mdape'].argmin()]

In [ ]:
tuning_results_all.sort_values('mape').head(20)

In [ ]:
tuning_results_all.corr().to_csv('fbprophet_corr_full.csv', index=False, mode='a', header=True) # ? first time add header')

In [ ]:
# actuals - labels
x = tuning_results_all.index
y = tuning_results_all["mape"]
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, name='mape'))
# Set title
fig.update_layout(
    title_text="Actual Open price - Labels",
)

fig.show()